In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import dask.array as da
import dask_ml
import datetime
import time
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf, pacf
import optuna
from lightgbm import LGBMClassifier
from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold, GroupKFold, train_test_split, KFold

import warnings
warnings.filterwarnings('ignore')
from warnings import simplefilter
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
pd.set_option("display.max_columns", None)
pd.set_option('use_inf_as_na', True)

In [2]:
book = pd.read_csv('log.txt')
book.rename(columns=lambda x: x.strip(), inplace=True)
book = book.fillna(0)

In [3]:
book

,sec,count,bid or ask,0 prise,0 size,1 prise,1 size,2 prise,2 size,3 prise,3 size,4 prise,4 size,5 prise,5 size,6 prise,6 size,7 prise,7 size,8 prise,8 size,9 prise,9 size,10 prise,10 size,11 prise,11 size,12 prise,12 size,13 prise,13 size,14 prise,14 size,15 prise,15 size,16 prise,16 size,17 prise,17 size,18 prise,18 size,19 prise,19 size,20 prise,20 size,21 prise,21 size,22 prise,22 size,23 prise,23 size,24 prise,24 size,25 prise,25 size,26 prise,26 size,27 prise,27 size,28 prise,28 size,29 prise,29 size,30 prise,30 size,31 prise,31 size,32 prise,32 size,33 prise,33 size,34 prise,34 size,35 prise,35 size,36 prise,36 size,37 prise,37 size,38 prise,38 size,39 prise,39 size,40 prise,40 size,41 prise,41 size,42 prise,42 size,43 prise,43 size,44 prise,44 size,45 prise,45 size,46 prise,46 size,47 prise,47 size,48 prise,48 size,49 prise,49 size,50 prise,50 size,51 prise,51 size,52 prise,52 size,53 prise,53 size,54 prise,54 size,55 prise,55 size,56 prise,56 size,57 prise,57 size,58 prise,58 size,59 prise,59 size,60 prise,60 size,61 prise,61 size,62 prise,62 size,63 prise,63 size,64 prise,64 size,65 prise,65 size,66 prise,66 size,67 prise,67 size,68 prise,68 size,69 prise,69 size,70 prise,70 size,71 prise,71 size,72 prise,72 size,73 prise,73 size,74 prise,74 size,75 prise,75 size,76 prise,76 size,77 prise,77 size,78 prise,78 size,79 prise,79 size,80 prise,80 size,81 prise,81 size,82 prise,82 size,83 prise,83 size,84 prise,84 size,85 prise,85 size,86 prise,86 size,87 prise,87 size,88 prise,88 size,89 prise,89 size,90 prise,90 size,91 prise,91 size,92 prise,92 size,93 prise,93 size,94 prise,94 size,95 prise,95 size,96 prise,96 size,97 prise,97 size,98 prise,98 size,99 prise,99 size,100 prise,100 size,101 prise,101 size,102 prise,102 size,103 prise,103 size,104 prise,104 size,105 prise,105 size,106 prise,106 size,107 prise,107 size,108 prise,108 size,109 prise,109 size,110 prise,110 size,111 prise,111 size,112 prise,112 size,113 prise,113 size,114 prise,114 size,115 prise,115 size,116 prise,116 size,117 prise,117 size,118 prise,118 size,119 prise,119 size,120 prise,120 size,121 prise,121 size,122 prise,122 size,123 prise,123 size,124 prise,124 size,125 prise,125 size,126 prise,126 size,127 prise,127 size,128 prise,128 size,129 prise,129 size,130 prise,130 size,131 prise,131 size,132 prise,132 size,133 prise,133 size,134 prise,134 size,135 prise,135 size,136 prise,136 size,137 prise,137 size,138 prise,138 size,139 prise,139 size,140 prise,140 size,141 prise,141 size,142 prise,142 size,143 prise,143 size,144 prise,144 size,145 prise,145 size,146 prise,146 size,147 prise,147 size,148 prise,148 size,149 prise,149 size,150 prise,150 size,151 prise,151 size,152 prise,152 size,153 prise,153 size,154 prise,154 size,155 prise,155 size,156 prise,156 size,157 prise,157 size,158 prise,158 size,159 prise,159 size,160 prise,160 size,161 prise,161 size,162 prise,162 size,163 prise,163 size,164 prise,164 size,165 prise,165 size,166 prise,166 size,167 prise,167 size,168 prise,168 size,169 prise,169 size,170 prise,170 size,171 prise,171 size,172 prise,172 size,173 prise,173 size,174 prise,174 size,175 prise,175 size,176 prise,176 size,177 prise,177 size,178 prise,178 size,179 prise,179 size,180 prise,180 size,181 prise,181 size,182 prise,182 size,183 prise,183 size,184 prise,184 size,185 prise,185 size,186 prise,186 size,187 prise,187 size,188 prise,188 size,189 prise,189 size,190 prise,190 size,191 prise,191 size,192 prise,192 size,193 prise,193 size,194 prise,194 size,195 prise,195 size,196 prise,196 size,197 prise,197 size,198 prise,198 size,199 prise,199 size,200 prise,200 size,201 prise,201 size,202 prise,202 size,203 prise,203 size,204 prise,204 size,205 prise,205 size,206 prise,206 size,207 prise,207 size,208 prise,208 size,209 prise,209 size,210 prise,210 size,211 prise,211 size,212 prise,212 size,213 prise,213 size,214 prise,214 size,215 prise,215 size,216 prise,216 size,217 prise,217 size,218 prise,218 size,219 prise,219 size,220 prise,220 size

### Preparation

In [4]:
ask_df = book[book['bid or ask']==' ASK']
bid_df = book[book['bid or ask']==' BID']

ask_df = ask_df.drop(['sec', 'bid or ask'], axis = 1)
bid_df = bid_df.drop(['sec', 'bid or ask'], axis = 1)

ask_df = ask_df.iloc[:, :53]
bid_df = bid_df.iloc[:, :53]

ask_bid = pd.merge(ask_df, bid_df, on='count', suffixes = ('_ask', '_bid'))

merged_mp = pd.merge(ask_df[['count', '0 prise', '0 size']], bid_df[['count', '0 prise', '0 size']], on = 'count', suffixes = ('_ask', '_bid'))

merged_mp['Midprice'] = (merged_mp['0 prise_bid']*merged_mp['0 size_bid'] + merged_mp['0 prise_ask']*merged_mp['0 size_ask'])/(merged_mp['0 size_bid']+merged_mp['0 size_ask'])
merged_mp['EWMA_midprice'] = merged_mp['Midprice'].ewm(span = 20).mean()


In [10]:
ask_bid

,count,0 prise_ask,0 size_ask,1 prise_ask,1 size_ask,2 prise_ask,2 size_ask,3 prise_ask,3 size_ask,4 prise_ask,4 size_ask,5 prise_ask,5 size_ask,6 prise_ask,6 size_ask,7 prise_ask,7 size_ask,8 prise_ask,8 size_ask,9 prise_ask,9 size_ask,10 prise_ask,10 size_ask,11 prise_ask,11 size_ask,12 prise_ask,12 size_ask,13 prise_ask,13 size_ask,14 prise_ask,14 size_ask,15 prise_ask,15 size_ask,16 prise_ask,16 size_ask,17 prise_ask,17 size_ask,18 prise_ask,18 size_ask,19 prise_ask,19 size_ask,20 prise_ask,20 size_ask,21 prise_ask,21 size_ask,22 prise_ask,22 size_ask,23 prise_ask,23 size_ask,24 prise_ask,24 size_ask,25 prise_ask,25 size_ask,0 prise_bid,0 size_bid,1 prise_bid,1 size_bid,2 prise_bid,2 size_bid,3 prise_bid,3 size_bid,4 prise_bid,4 size_bid,5 prise_bid,5 size_bid,6 prise_bid,6 size_bid,7 prise_bid,7 size_bid,8 prise_bid,8 size_bid,9 prise_bid,9 size_bid,10 prise_bid,10 size_bid,11 prise_bid,11 size_bid,12 prise_bid,12 size_bid,13 prise_bid,13 size_bid,14 prise_bid,14 size_bid,15 prise_bid,15 size_bid,16 prise_bid,16 size_bid,17 prise_bid,17 size_bid,18 prise_bid,18 size_bid,19 prise_bid,19 size_bid,20 prise_bid,20 size_bid,21 prise_bid,21 size_bid,22 prise_bid,22 size_bid,23 prise_bid,23 size_bid,24 prise_bid,24 size_bid,25 prise_bid,25 size_bid
0,0,8.464,50000,8.465,180000,8.466,249000,8.467,900000,8.468,1450000,8.469,5220000,8.47,3450000,8.471,11700000,8.472,3591000,8.473,800000,8.474,367000,8.479,200000.0,8.510,2100000,8.518,2000000.0,8.539,1212000.0,8.541,2525000.0,8.549,2020000.0,8.550,1010000.0,8.553,2626000.0,8.554,4000000.0,8.558,2828000.0,8.559,1010000.0,8.560,1212000.0,8.562,4000000.0,8.564,2222000.0,8.565,1515000.0,8.460,50000,8.459,400000,8.458,50000,8.457,250000,8.456,213000,8.455,54000,8.454,50000,8.453,200000,8.452,50000,8.451,213000,8.450,50000,8.449,6.000000e+05,8.448,50000,8.447,4.000000e+05,8.446,2.130000e+05,8.445,7.800000e+05,8.444,37000.0,8.443,600000.0,8.442,445000.0,8.441,1760000.0,8.440,1.800000e+06,8.439,11100000.0,8.438,3980000.0,8.437,21291000.0,8.436,3675000.0,8.435,20735000.0
1,1,8.464,50000,8.465,180000,8.466,249000,8.467,900000,8.468,1450000,8.469,5220000,8.47,3450000,8.471,11700000,8.472,3591000,8.473,800000,8.474,367000,8.479,200000.0,8.510,2100000,8.518,2000000.0,8.524,6000.0,8.539,1212000.0,8.541,2525000.0,8.549,2020000.0,8.550,1010000.0,8.553,2626000.0,8.554,4000000.0,8.558,2828000.0,8.559,1010000.0,8.560,1212000.0,8.562,4000000.0,8.564,2222000.0,8.460,50000,8.459,400000,8.458,50000,8.457,250000,8.456,213000,8.455,54000,8.454,50000,8.453,200000,8.452,50000,8.451,213000,8.450,50000,8.449,6.000000e+05,8.448,50000,8.447,4.000000e+05,8.446,2.130000e+05,8.445,7.800000e+05,8.444,37000.0,8.443,600000.0,8.442,445000.0,8.441,1760000.0,8.440,1.800000e+06,8.439,11100000.0,8.438,3980000.0,8.437,21291000.0,8.436,3675000.0,8.435,20735000.0
2,2,8.464,50000,8.465,180000,8.466,249000,8.467,900000,8.468,1450000,8.469,5220000,8.47,3450000,8.471,11700000,8.472,3591000,8.473,800000,8.474,367000,8.479,200000.0,8.510,2100000,8.518,2000000.0,8.524,6000.0,8.539,1212000.0,8.541,2525000.0,8.549,2020000.0,8.550,1010000.0,8.553,2626000.0,8.554,4000000.0,8.558,2828000.0,8.559,1010000.0,8.560,1212000.0,8.562,4000000.0,8.564,2222000.0,8.460,50000,8.459,400000,8.458,50000,8.457,250000,8.456,213000,8.455,54000,8.454,50000,8.453,200000,8.452,50000,8.451,213000,8.450,50000,8.449,6.000000e+05,8.448,50000,8.447,4.000000e+05,8.446,2.130000e+05,8.445,7.800000e+05,8.444,37000.0,8.443,600000.0,8.442,445000.0,8.441,1760000.0,8.440,1.800000e+06,8.439,11100000.0,8.438,3980000.0,8.437,21291000.0,8.436,3675000.0,8.435,20735000.0
3,3,8.464,50000,8.465,180000,8.466,249000,8.467,900000,8.468,1450000,8.469,5220000,8.47,3450000,8.471,11700000,8.472,3591000,8.473,800000,8.474,367000,8.479,200000.0,8.510,2100000,8.518,2000000.0,8.524,6000.0,8.539,1212000.0,8.541,2525000.0,8.549,2020000.0,8.550,1010000.0,8.553,2626000.0,8.554,4000000.0,8.558,2828000.0,8.559,1010000.0,8.560,1212000.0,8.562,4000000.0,8.564,2222000.0,8.460,50000,8.459,400000,8.45

In [8]:
merged_mp['target'] = merged_mp['EWMA_midprice'].shift(-10)
merged_mp['target_returns'] = np.log(merged_mp['target']) - np.log(merged_mp['EWMA_midprice'])

In [9]:
merged_mp

,count,0 prise_ask,0 size_ask,0 prise_bid,0 size_bid,Midprice,EWMA_midprice,target,target_returns
0,0,8.464,50000,8.460,50000,8.462000,8.462000,8.462000,0.000000
1,1,8.464,50000,8.460,50000,8.462000,8.462000,8.462000,0.000000
2,2,8.464,50000,8.460,50000,8.462000,8.462000,8.462131,0.000015
3,3,8.464,50000,8.460,50000,8.462000,8.462000,8.462266,0.000031
4,4,8.464,50000,8.460,50000,8.462000,8.462000,8.462380,0.000045
...,...,...,...,...,...,...,...,...,...
36215,36215,8.404,1034000,8.479,174360000,8.478558,8.478558,NaN,NaN
36216,36216,8.404,1034000,8.479,174360000,8.478558,8.478558,NaN,NaN
36217,36217,8.404,1034000,8.479,174360000,8.478558,8.478558,NaN,NaN
36218,36218,8.404,1034000,8.479,174360000,8.478558,8.478558,NaN,NaN


### Bid ask spread

In [11]:
merged_mp['Bid-Ask Spread'] =  merged_mp['0 prise_ask']/merged_mp['0 prise_bid'] - 1
merged_mp['Bid-Ask Spread_cumulative'] = merged_mp['Bid-Ask Spread'].rolling(10).sum()

### Returns / Momentum

In [12]:
merged_mp['Momentum_100'] = np.log(merged_mp['EWMA_midprice']).diff(100)
merged_mp['Momentum_200'] = np.log(merged_mp['EWMA_midprice']).diff(200)
merged_mp['Momentum_500'] = np.log(merged_mp['EWMA_midprice']).diff(500)

### ACF

In [13]:
mu = merged_mp['Momentum_100'].rolling(1000).mean()
lag_100 = merged_mp['Momentum_100'].shift(100)
lag_150 = merged_mp['Momentum_100'].shift(150)
lag_500 = merged_mp['Momentum_100'].shift(500)
std_closing = merged_mp['Momentum_100'].rolling(1000).std()
std_lag_100 = lag_100.rolling(100).std()
std_lag_150 = lag_150.rolling(150).std()
std_lag_500 = lag_500.rolling(500).std()

merged_mp['ACF_lag_100'] = ((merged_mp['Momentum_100'] - mu) * (lag_100 - mu)).rolling(100).mean()/(std_closing * std_lag_100)
merged_mp['ACF_lag_150'] = ((merged_mp['Momentum_100'] - mu) * (lag_150 - mu)).rolling(150).mean()/(std_closing * std_lag_150)
merged_mp['ACF_lag_500'] = ((merged_mp['Momentum_100'] - mu) * (lag_500 - mu)).rolling(500).mean()/(std_closing * std_lag_500)

merged_mp['ACF_lag_100'] = merged_mp['ACF_lag_100'].fillna(0)
merged_mp['ACF_lag_150'] = merged_mp['ACF_lag_150'].fillna(0)
merged_mp['ACF_lag_500'] = merged_mp['ACF_lag_500'].fillna(0)

### Order book IMB

In [14]:
def calculate_VI(df):
    ask_vol = 0
    bid_vol = 0
    for i in range(0, 25):
        ask_vol = ask_vol + df[f'{i} prise_ask'] * df[f'{i} size_ask']
        bid_vol = bid_vol + df[f'{i} prise_bid'] * df[f'{i} size_bid']

    df['VI'] = (bid_vol - ask_vol) / (bid_vol + ask_vol)
    return df['VI']

merged_mp['VI'] = calculate_VI(ask_bid)

### IMB feature

In [15]:
def IMB_feature_pandas(df):
    def calc_avg_price(prices, amounts, target_volume):
        total_value = np.zeros_like(target_volume)
        cum_volume = np.zeros_like(target_volume)
        
        for p, a in zip(prices.T, amounts.T):
            remaining_volume = np.maximum(target_volume - cum_volume, 0)
            add_volume = np.minimum(a, remaining_volume)
            total_value += p * add_volume
            cum_volume += add_volume
            
        return total_value / target_volume

    ask_prices = df[[f'{i} prise_ask' for i in range(25)]].values
    ask_amounts = df[[f'{i} size_ask' for i in range(25)]].values
    bid_prices = df[[f'{i} prise_bid' for i in range(25)]].values
    bid_amounts = df[[f'{i} size_bid' for i in range(25)]].values

    asks_mask = ask_prices <= df[f'{0} prise_ask'].values[:, None] * 1.01
    bids_mask = bid_prices >= df[f'{0} prise_bid'].values[:, None] * 0.99

    # Check NaN count
    print("NaN count in asks_mask:", np.isnan(asks_mask).sum())
    print("NaN count in bids_mask:", np.isnan(bids_mask).sum())

    filtered_ask_prices = np.where(asks_mask, ask_prices, np.nan)
    filtered_ask_amounts = np.where(asks_mask, ask_amounts, np.nan)
    filtered_bid_prices = np.where(bids_mask, bid_prices, np.nan)
    filtered_bid_amounts = np.where(bids_mask, bid_amounts, np.nan)

    # Check NaN count
    print("NaN count in filtered_ask_prices:", np.isnan(filtered_ask_prices).sum())
    print("NaN count in filtered_ask_amounts:", np.isnan(filtered_ask_amounts).sum())
    print("NaN count in filtered_bid_prices:", np.isnan(filtered_bid_prices).sum())
    print("NaN count in filtered_bid_amounts:", np.isnan(filtered_bid_amounts).sum())

    mean_ask_amount = np.nanmean(filtered_ask_amounts, axis=1)
    mean_bid_amount = np.nanmean(filtered_bid_amounts, axis=1)

    N = (mean_ask_amount + mean_bid_amount) / 2
    N[N == 0] = 1

    avg_ask_price = calc_avg_price(filtered_ask_prices, filtered_ask_amounts, N)
    avg_bid_price = calc_avg_price(filtered_bid_prices, filtered_bid_amounts, N)

    # Check NaN count
    print("NaN count in avg_ask_price:", np.isnan(avg_ask_price).sum())
    print("NaN count in avg_bid_price:", np.isnan(avg_bid_price).sum())

    IMB_ask = (avg_ask_price / df[f'{0} prise_ask'].values - 1)
    IMB_bid = (df[f'{0} prise_bid'].values / avg_bid_price - 1)

    # Check NaN count
    print("NaN count in IMB_ask:", np.isnan(IMB_ask).sum())
    print("NaN count in IMB_bid:", np.isnan(IMB_bid).sum())

    result = pd.DataFrame(np.stack([avg_ask_price, avg_bid_price, IMB_ask, IMB_bid, df['count']], axis=1), 
                          columns=["avg_ask_price", "avg_bid_price", "IMB_ask", "IMB_bid", 'count'])
    
    
    return result

result  = IMB_feature_pandas(ask_bid)

NaN count in asks_mask: 0
NaN count in bids_mask: 0
NaN count in filtered_ask_prices: 217
NaN count in filtered_ask_amounts: 217
NaN count in filtered_bid_prices: 0
NaN count in filtered_bid_amounts: 0
NaN count in avg_ask_price: 49
NaN count in avg_bid_price: 0
NaN count in IMB_ask: 49
NaN count in IMB_bid: 0


In [16]:
result['IMB_ask_change'] = (result['IMB_ask'].diff(100)/result['IMB_ask'].shift(100)).fillna(0)
result['IMB_bid_change'] = (result['IMB_bid'].diff(100)/result['IMB_bid'].shift(100)).fillna(0)
result['IMB_ask_change_cum'] = result['IMB_ask_change'].rolling(100).sum()
result['IMB_bid_change_cum'] = result['IMB_bid_change'].rolling(100).sum()

result = result.drop(['avg_ask_price', 'avg_bid_price'], axis = 1)

merged_mp = pd.merge(merged_mp, result, on='count')

In [17]:
merged_mp

,count,0 prise_ask,0 size_ask,0 prise_bid,0 size_bid,Midprice,EWMA_midprice,target,target_returns,Bid-Ask Spread,Bid-Ask Spread_cumulative,Momentum_100,Momentum_200,Momentum_500,ACF_lag_100,ACF_lag_150,ACF_lag_500,VI,IMB_ask,IMB_bid,IMB_ask_change,IMB_bid_change,IMB_ask_change_cum,IMB_bid_change_cum
0,0,8.464,50000,8.460,50000,8.462000,8.462000,8.462000,0.000000,0.000473,NaN,NaN,NaN,NaN,0.0,0.0,0.0,-0.085992,NaN,0.000708,0.0,0.0,NaN,NaN
1,1,8.464,50000,8.460,50000,8.462000,8.462000,8.462000,0.000000,0.000473,NaN,NaN,NaN,NaN,0.0,0.0,0.0,-0.066125,NaN,0.000688,0.0,0.0,NaN,NaN
2,2,8.464,50000,8.460,50000,8.462000,8.462000,8.462131,0.000015,0.000473,NaN,NaN,NaN,NaN,0.0,0.0,0.0,-0.066125,NaN,0.000688,0.0,0.0,NaN,NaN
3,3,8.464,50000,8.460,50000,8.462000,8.462000,8.462266,0.000031,0.000473,NaN,NaN,NaN,NaN,0.0,0.0,0.0,-0.066125,NaN,0.000688,0.0,0.0,NaN,NaN
4,4,8.464,50000,8.460,50000,8.462000,8.462000,8.462380,0.000045,0.000473,NaN,NaN,NaN,NaN,0.0,0.0,0.0,-0.066125,NaN,0.000688,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36215,36215,8.404,1034000,8.479,174360000,8.478558,8.478558,NaN,NaN,-0.008845,-0.088454,0.0,0.0,0.0,0.0,0.0,0.0,0.680625,0.001137,0.000054,0.0,0.0,2.560195e-14,-2.574077e-13
36216,36216,8.404,1034000,8.479,174360000,8.478558,8.478558,NaN,NaN,-0.008845,-0.088454,0.0,0.0,0.0,0.0,0.0,0.0,0.680625,0.001137,0.000054,0.0,0.0,2.560195e-14,-2.574077e-13
36217,36217,8.404,1034000,8.479,174360000,8.478558,8.478558,NaN,NaN,-0.008845,-0.088454,0.0,0.0,0.0,0.0,0.0,0.0,0.680625,0.001137,0.000054,0.0,0.0,2.560195e-14,-2.574077e-13
36218,36218,8.404,1034000,8.479,174360000,8.478558,8.478558,NaN,NaN,-0.008845,-0.088454,0.0,0.0,0.0,0.0,0.0,0.0,0.680625,0.001137,0.000054,0.0,0.0,2.560195e-14,-2.574077e-13


### Realized Volatility

In [18]:
merged_mp['Realized_std_100'] = merged_mp['Momentum_100'].rolling(100).std()
merged_mp['Realized_std_200'] = merged_mp['Momentum_100'].rolling(200).std()
merged_mp['Realized_std_500'] = merged_mp['Momentum_100'].rolling(500).std()
merged_mp['Realized_std_1000'] = merged_mp['Momentum_100'].rolling(1000).std()

### Jump Variation v2

In [19]:
merged_mp['JV_100'] = (merged_mp['Momentum_100']**2).rolling(100).sum() - (np.pi/2) * np.abs(merged_mp['Momentum_100']) * np.abs(merged_mp['Momentum_100'].shift(100))
merged_mp['JV_200'] = (merged_mp['Momentum_100']**2).rolling(200).sum() - (np.pi/2) * np.abs(merged_mp['Momentum_100']) * np.abs(merged_mp['Momentum_100'].shift(200))
merged_mp['JV_500'] = (merged_mp['Momentum_100']**2).rolling(500).sum() - (np.pi/2) * np.abs(merged_mp['Momentum_100']) * np.abs(merged_mp['Momentum_100'].shift(500))


In [20]:
merged_mp = merged_mp.dropna()
merged_mp = merged_mp.drop(['0 prise_ask', '0 size_ask', '0 prise_bid', '0 size_bid', 'Midprice','count'], axis = 1)
merged_mp

,EWMA_midprice,target,target_returns,Bid-Ask Spread,Bid-Ask Spread_cumulative,Momentum_100,Momentum_200,Momentum_500,ACF_lag_100,ACF_lag_150,ACF_lag_500,VI,IMB_ask,IMB_bid,IMB_ask_change,IMB_bid_change,IMB_ask_change_cum,IMB_bid_change_cum,Realized_std_100,Realized_std_200,Realized_std_500,Realized_std_1000,JV_100,JV_200,JV_500
1099,8.478996,8.478996,0.0,-0.001769,-0.017691,2.806644e-13,6.252358e-09,0.000033,0.0,0.0,0.0,0.735363,0.000965,0.000000,-0.016333,0.0,-1.568368e+00,-2.557954e-13,1.193925e-09,0.000002,0.000008,2.583693e-04,1.764094e-16,1.291761e-09,5.138253e-08
1100,8.478996,8.478996,0.0,-0.001769,-0.017691,2.535749e-13,5.656895e-09,0.000033,0.0,0.0,0.0,0.735462,0.000965,0.000000,-0.016254,0.0,-1.590887e+00,-2.557954e-13,1.080225e-09,0.000002,0.000008,2.567202e-04,1.444123e-16,1.249639e-09,5.138252e-08
1101,8.478996,8.478996,0.0,-0.001769,-0.017691,2.291500e-13,5.118143e-09,0.000033,0.0,0.0,0.0,0.735462,0.000965,0.000000,-0.016254,0.0,-1.613323e+00,-2.557954e-13,9.773532e-10,0.000002,0.000008,2.550560e-04,1.182197e-16,1.207524e-09,5.138252e-08
1102,8.478996,8.478996,0.0,-0.001769,-0.017691,2.073897e-13,4.630701e-09,0.000033,0.0,0.0,0.0,0.735462,0.000965,0.000000,-0.016380,0.0,-1.635920e+00,-2.557954e-13,8.842799e-10,0.000002,0.000008,2.533764e-04,9.677855e-17,1.165417e-09,5.138252e-08
1103,8.478996,8.478996,0.0,-0.001769,-0.017691,1.874056e-13,4.189681e-09,0.000033,0.0,0.0,0.0,0.735684,0.000965,0.000000,-0.016989,0.0,-1.659705e+00,-2.557954e-13,8.000716e-10,0.000002,0.000008,2.516811e-04,7.922697e-17,1.124060e-09,5.138251e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36205,8.478558,8.478558,0.0,-0.008845,-0.088454,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,0.0,0.680625,0.001137,0.000054,0.000000,0.0,2.560195e-14,-2.574077e-13,8.642269e-12,0.000000,0.000000,8.658063e-12,2.447088e-20,3.620372e-21,1.758980e-20
36206,8.478558,8.478558,0.0,-0.008845,-0.088454,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,0.0,0.680625,0.001137,0.000054,0.000000,0.0,2.560195e-14,-2.574077e-13,8.642269e-12,0.000000,0.000000,8.658063e-12,2.447088e-20,3.620372e-21,1.758980e-20
36207,8.478558,8.478558,0.0,-0.008845,-0.088454,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,0.0,0.680625,0.001137,0.000054,0.000000,0.0,2.560195e-14,-2.574077e-13,8.642269e-12,0.000000,0.000000,8.658063e-12,2.447088e-20,3.620372e-21,1.758980e-20
36208,8.478558,8.478558,0.0,-0.008845,-0.088454,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,0.0,0.680625,0.001137,0.000054,0.000000,0.0,2.560195e-14,-2.574077e-13,8.642269e-12,0.000000,0.000000,8.658063e-12,2.447088e-20,3.620372e-21,1.758980e-20


### Model

In [ ]:
# Necessary Libraries
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

X = merged_mp.drop(['target_returns', 'target', 'EWMA_midprice'], axis = 1)
y = merged_mp['target_returns']

# Time-based split
# Using 70-30 split. Modify according to your requirements
train_size = int(len(X) * 0.7)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Now within the training set perform cross validation and parameter tuning
# Define the model
model = lgb.LGBMRegressor(random_state=42, n_jobs = -1) # replace with your model

# Define the parameters to tune
params_grid = {'max_depth':[5, 8, 11, 13],
               'num_iterations':[100, 200, 300],
               'colsample_bytree':[0.8, 0.9],
               'subsample':[0.8, 0.9],
               'reg_lambda':[0.001, 0.01, 0.1],
               'learning_rate':[0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
              }

# Define time-series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Setup the grid search
gsearch = GridSearchCV(estimator=model,
                       cv=tscv,
                       param_grid=params_grid,
                       scoring='neg_mean_squared_error') # modify with your scoring method

# Fit the grid search
gsearch.fit(X_train, y_train)

# Get the best model
best_model = gsearch.best_estimator_

# Validate the model using the test set
predictions = best_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)

print(f'Best Model: {best_model}')
print(f'Test MSE: {mse}')


In [ ]:
X_test['predictions'] = predictions
X_test = X_test.join(merged_mp[['target_returns', 'target', 'EWMA_midprice']])
X_test['predictions_price'] = (1+X_test['predictions']) * X_test['EWMA_midprice']
rmse = float(np.sqrt(mean_squared_error(X_test['predictions_price'], X_test['target'])))
print("{:.10f}".format(rmse))